# Trabajo Final Grupo 4


### Dataset utilizado:
**OLIVES**: Etiquetas Oftálmicas para Investigar Semántica Visual Ocular  
URL: https://zenodo.org/records/7105232

## 1. Import Required Libraries
Importamos las librerías necesarias para el manejo de entrada de usuario y operaciones del sistema.

In [38]:
# Import Required Libraries
import os
import sys
import json
import logging
from typing import Optional, List, Dict, Any
from datetime import datetime

# Configurar el path para importar el paquete medical_image_manager
project_root = os.path.dirname(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Importar módulos del proyecto
from medical_image_manager import ImageManager, DataLoader, MedicalImage, ImageMetadata
from medical_image_manager.utils.validators import DataValidator

# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("Librerías importadas correctamente")
print("Directorio de trabajo:", os.getcwd())

Librerías importadas correctamente
Directorio de trabajo: C:\Users\EYNAR\Desktop\MAESTRAI_IA_MOD1\notebooks


## 2. Initialize System Components
Inicializamos los componentes principales del sistema de gestión de imágenes médicas.

In [39]:
# Initialize System Components
class MedicalImageNotebookApp:
    """
    Aplicación de Gestión de Imágenes Médicas para Jupyter Notebook
    Similar a main.py pero adaptada para uso interactivo en notebook
    """
    
    def __init__(self):
        """Inicializar la aplicación"""
        self.image_manager = ImageManager()
        self.data_loader = DataLoader()
        self.validator = DataValidator()
        self.logger = logging.getLogger(__name__)
        
        print("=" * 60)
        print("     Medical Image Management System")
        print("     Trabajo Final Grupo 4")
        print("=" * 60)
        print("Sistema inicializado correctamente")

# Crear instancia global de la aplicación
app = MedicalImageNotebookApp()

     Medical Image Management System
     Trabajo Final Grupo 4
Sistema inicializado correctamente


## 3. Define Menu Options Structure
Creamos la estructura de datos para almacenar las opciones del menú y sus descripciones.

In [40]:
# Define Menu Options Structure
MENU_OPTIONS = {
    "1": {
        "title": "Registrar Nueva Imagen",
        "description": "Registra una nueva imagen médica en el sistema"
    },
    "2": {
        "title": "Ver Imágenes Registradas", 
        "description": "Visualiza todas las imágenes registradas en el sistema"
    },
    "3": {
        "title": "Modificar Metadatos de Imagen",
        "description": "Modifica los metadatos de una imagen existente"
    },
    "4": {
        "title": "Eliminar Imagen",
        "description": "Elimina una imagen del sistema"
    },
    "5": {
        "title": "Buscar Imágenes", 
        "description": "Busca imágenes por criterios específicos"
    },
    "6": {
        "title": "Cargar Dataset de Zenodo",
        "description": "Descarga y carga el dataset OLIVES desde Zenodo"
    },
    "7": {
        "title": "Salir",
        "description": "Termina la aplicación"
    }
}

print("Opciones del menú configuradas:")
for key, option in MENU_OPTIONS.items():
    print(f"  {key}. {option['title']}")

Opciones del menú configuradas:
  1. Registrar Nueva Imagen
  2. Ver Imágenes Registradas
  3. Modificar Metadatos de Imagen
  4. Eliminar Imagen
  5. Buscar Imágenes
  6. Cargar Dataset de Zenodo
  7. Salir


## 4. Create Menu Display Function
Implementamos una función que muestra las opciones del menú al usuario de forma formateada.

In [41]:
# Create Menu Display Function
def display_main_menu():
    """Mostrar las opciones del menú principal de forma formateada."""
    print("\n" + "=" * 60)
    print("            MENU PRINCIPAL")
    print("=" * 60)
    
    for key, option in MENU_OPTIONS.items():
        print(f"{key}. {option['title']}")
        if key in ["4", "6"]:  # Agregar separadores para mejor organización
            print("   " + "-" * 25)
    
    print("=" * 60)

def get_menu_choice():
    """Obtener y validar la elección del usuario."""
    while True:
        display_main_menu()
        choice = input(f"\nSeleccione una opción (1-{len(MENU_OPTIONS)}): ").strip()
        
        if choice in MENU_OPTIONS:
            return choice
        else:
            print(f"Opción inválida: '{choice}'. Por favor intente de nuevo.")
            print(f"Opciones válidas: {', '.join(MENU_OPTIONS.keys())}")

# Probar la función de display
display_main_menu()


            MENU PRINCIPAL
1. Registrar Nueva Imagen
2. Ver Imágenes Registradas
3. Modificar Metadatos de Imagen
4. Eliminar Imagen
   -------------------------
5. Buscar Imágenes
6. Cargar Dataset de Zenodo
   -------------------------
7. Salir


## 5. Implement Input Validation Functions
Creamos funciones para validar la entrada del usuario y manejar selecciones inválidas de manera elegante.

In [42]:
# Implement Input Validation Functions
def validate_file_path(file_path: str) -> bool:
    """Validar si la ruta del archivo existe y es válida."""
    if not file_path or not file_path.strip():
        print("La ruta del archivo no puede estar vacía.")
        return False
    
    if not os.path.exists(file_path):
        print(f"Archivo no encontrado: {file_path}")
        return False
    
    return True

def validate_yes_no_input(prompt: str) -> bool:
    """Validar entrada sí/no del usuario."""
    while True:
        response = input(f"{prompt} (s/n): ").strip().lower()
        if response in ['s', 'sí', 'si', 'y', 'yes']:
            return True
        elif response in ['n', 'no']:
            return False
        else:
            print("Respuesta inválida. Por favor ingrese 's' para sí o 'n' para no.")

def validate_numeric_choice(prompt: str, min_val: int, max_val: int) -> int:
    """Validar entrada numérica dentro de un rango."""
    while True:
        try:
            choice = int(input(f"{prompt} ({min_val}-{max_val}): "))
            if min_val <= choice <= max_val:
                return choice
            else:
                print(f"Número fuera de rango. Ingrese un número entre {min_val} y {max_val}.")
        except ValueError:
            print("Por favor ingrese un número válido.")

def safe_input(prompt: str, required: bool = False) -> str:
    """Entrada segura con validación opcional de campos requeridos."""
    while True:
        value = input(prompt).strip()
        if not required or value:
            return value
        print("Este campo es requerido. Por favor ingrese un valor.")

print("Funciones de validación de entrada configuradas")

Funciones de validación de entrada configuradas


## 6. Handle Menu Selection Logic
Implementamos la lógica para procesar las selecciones del usuario y ejecutar las acciones correspondientes.

In [ ]:
# Handle Menu Selection Logic - Función 1: Registrar Nueva Imagen
def register_image_menu():
    """Manejar el registro de imágenes."""
    print("\n" + "=====================================" )
    print("REGISTRAR NUEVA IMAGEN")
    print("=====================================")
    
    file_path = safe_input("Ingrese la ruta del archivo de imagen: ", required=True)
    
    if not validate_file_path(file_path):
        return
    
    try:
        # Preguntar si el usuario quiere agregar metadatos
        add_metadata = validate_yes_no_input("¿Agregar metadatos personalizados?")
        
        metadata = None
        if add_metadata:
            metadata = create_metadata_interactive()
        
        # Registrar la imagen
        load_pixels = validate_yes_no_input("¿Cargar datos de píxeles en memoria?")
        image_id = app.image_manager.register_image(
            file_path=file_path,
            metadata=metadata,
            load_pixel_data=load_pixels
        )
        
        print(f"\n¡Imagen registrada exitosamente!")
        print(f"ID de Imagen: {image_id}")
        
    except Exception as e:
        print(f"Error al registrar imagen: {e}")

def create_metadata_interactive() -> ImageMetadata:
    """Crear metadatos de forma interactiva."""
    metadata = ImageMetadata()
    
    print("\n--- CREAR METADATOS ---")
    
    # Información del paciente
    patient_id = safe_input("Patient ID: ")
    if patient_id:
        metadata.update_patient_info(patient_id=patient_id)
    
    patient_name = safe_input("Patient Name: ")
    if patient_name:
        metadata.update_patient_info(patient_name=patient_name)
    
    # Información del estudio
    modality = safe_input("Modality (CT, MRI, PET, etc.): ")
    if modality:
        metadata.update_study_info(modality=modality)
    
    study_desc = safe_input("Study Description: ")
    if study_desc:
        metadata.update_study_info(study_description=study_desc)
    
    # Campos personalizados
    while True:
        add_custom = validate_yes_no_input("¿Agregar campo personalizado?")
        if not add_custom:
            break
        
        key = safe_input("Nombre del campo: ", required=True)
        value = safe_input("Valor del campo: ", required=True)
        
        metadata.add_custom_field(key, value)
    
    return metadata

print("Función de registro de imágenes configurada")

Función de registro de imágenes configurada


In [ ]:
# Handle Menu Selection Logic - Función 2: Ver Imágenes Registradas
def view_images_menu():
    """Manejar la visualización de imágenes registradas."""
    print("\n" + "===================================")
    print("VER IMAGENES REGISTRADAS")
    print("===================================")
    
    images = app.image_manager.list_images()
    
    if not images:
        print("No hay imágenes registradas.")
        return
    
    print(f"\nSe encontraron {len(images)} imágenes registradas:\n")
    
    for i, image in enumerate(images, 1):
        print(f"{i}. {image.filename}")
        print(f"   ID: {image.image_id}")
        print(f"   Formato: {image.format_type}")
        print(f"   Tamaño: {image.file_size or 'Desconocido'} bytes")
        print(f"   Creado: {image.creation_date.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   Datos de píxeles: {'Sí' if image.pixel_data is not None else 'No'}")
        print()
    
    # Preguntar si el usuario quiere ver detalles de una imagen específica
    view_details = validate_yes_no_input("¿Ver detalles de una imagen específica?")
    
    if view_details:
        choice = validate_numeric_choice(f"Ingrese número de imagen", 1, len(images)) - 1
        display_image_details(images[choice])

def display_image_details(image: MedicalImage):
    """Mostrar información detallada de una imagen."""
    print(f"\n" + "===================================")
    print(f"DETALLES DE IMAGEN: {image.filename}")
    print("===================================")
    
    info = image.get_image_info()
    
    for key, value in info.items():
        if key != 'metadata_fields':
            display_key = key.replace('_', ' ').title()
            print(f"{display_key}: {value}")
    
    # Mostrar metadatos
    if image.metadata:
        print("\nMetadatos:")
        for key, value in image.metadata.items():
            # Truncar valores largos
            str_value = str(value)
            if len(str_value) > 100:
                str_value = str_value[:97] + "..."
            print(f"  {key}: {str_value}")
    
    # Estado de validación
    is_valid = app.validator.validate_image(image)
    status = 'Válida' if is_valid else 'Inválida'
    print(f"\nEstado de Validación: {status}")
    
    if not is_valid:
        errors = app.validator.get_validation_errors(image)
        print("Errores de Validación:")
        for error in errors:
            print(f"  - {error}")

print("Función de visualización de imágenes configurada")

Función de visualización de imágenes configurada


In [ ]:
# Handle Menu Selection Logic - Función 3: Modificar Metadatos
def modify_metadata_menu():
    """Manejar la modificación de metadatos."""
    print("\n" + "======================" )
    print("MODIFICAR METADATOS DE IMAGEN")
    print("=========================")
    
    images = app.image_manager.list_images()
    if not images:
        print("No hay imágenes registradas.")
        return
    
    # Seleccionar imagen
    print("Seleccione una imagen para modificar:")
    for i, image in enumerate(images, 1):
        print(f"{i}. {image.filename} (ID: {image.image_id[:8]}...)")
    
    choice = validate_numeric_choice(f"Ingrese número de imagen", 1, len(images)) - 1
    selected_image = images[choice]
    
    # Opciones de modificación
    print(f"\nOpciones de modificación para: {selected_image.filename}")
    print("1. Agregar/Actualizar campo de metadatos")
    print("2. Eliminar campo de metadatos")
    print("3. Actualización masiva desde JSON")
    
    mod_choice = validate_numeric_choice("Seleccione opción", 1, 3)
    
    try:
        if mod_choice == 1:
            # Agregar/Actualizar campo
            key = safe_input("Ingrese clave de metadatos: ", required=True)
            value = safe_input("Ingrese valor de metadatos: ", required=True)
            
            # Intentar convertir al tipo apropiado
            try:
                if value.isdigit():
                    value = int(value)
                elif '.' in value and value.replace('.', '').isdigit():
                    value = float(value)
            except:
                pass
            
            success = app.image_manager.update_image_metadata(
                selected_image.image_id, {key: value}
            )
            
            if success:
                print(f"Metadatos actualizados: {key} = {value}")
            else:
                print("Error al actualizar metadatos.")
        
        elif mod_choice == 2:
            # Eliminar campo
            if selected_image.metadata:
                print("\nClaves de metadatos existentes:")
                keys = list(selected_image.metadata.keys())
                for i, key in enumerate(keys, 1):
                    print(f"{i}. {key}")
                
                key_choice = validate_numeric_choice(f"Seleccione clave a eliminar", 1, len(keys)) - 1
                key_to_remove = keys[key_choice]
                selected_image.remove_metadata(key_to_remove)
                print(f"Clave de metadatos eliminada: {key_to_remove}")
            else:
                print("No hay metadatos para eliminar.")
        
        elif mod_choice == 3:
            # Actualización desde JSON
            json_str = safe_input("Ingrese JSON de metadatos (o ruta de archivo): ", required=True)
            
            if os.path.exists(json_str):
                with open(json_str, 'r') as f:
                    json_str = f.read()
            
            try:
                metadata_dict = json.loads(json_str)
                
                success = app.image_manager.update_image_metadata(
                    selected_image.image_id, metadata_dict
                )
                
                if success:
                    print("Metadatos actualizados desde JSON.")
                else:
                    print("Error al actualizar metadatos.")
                    
            except json.JSONDecodeError:
                print("Formato JSON inválido.")
                
    except Exception as e:
        print(f"Error modificando metadatos: {e}")

print("Función de modificación de metadatos configurada")

Función de modificación de metadatos configurada


In [ ]:
# Handle Menu Selection Logic - Funciones 4 y 5: Eliminar y Buscar Imágenes
def delete_image_menu():
    """Manejar la eliminación de imágenes."""
    print("\n" + "===================================")
    print("ELIMINAR IMAGEN")
    print("===================================")
    
    images = app.image_manager.list_images()
    if not images:
        print("No hay imágenes registradas.")
        return
    
    # Seleccionar imagen para eliminar
    print("Seleccione una imagen para eliminar:")
    for i, image in enumerate(images, 1):
        print(f"{i}. {image.filename} (ID: {image.image_id[:8]}...)")
    
    choice = validate_numeric_choice(f"Ingrese número de imagen", 1, len(images)) - 1
    selected_image = images[choice]
    
    # Confirmación
    print(f"\n¿Está seguro de eliminar '{selected_image.filename}'?")
    confirm = validate_yes_no_input("Confirmar eliminación")
    
    if confirm:
        try:
            success = app.image_manager.delete_image(selected_image.image_id)
            
            if success:
                print("Imagen eliminada exitosamente.")
            else:
                print("Error al eliminar imagen.")
        except Exception as e:
            print(f"Error eliminando imagen: {e}")
    else:
        print("Eliminación cancelada.")

def search_images_menu():
    """Manejar la búsqueda de imágenes."""
    print("\n" + "===================================")
    print("BUSCAR IMAGENES")
    print("===================================")
    
    query = safe_input("Ingrese consulta de búsqueda: ", required=True)
    
    # Opciones de búsqueda
    print("\nBuscar en:")
    print("1. Todos los campos")
    print("2. Solo nombre de archivo")
    print("3. Campos específicos de metadatos")
    
    search_choice = validate_numeric_choice("Seleccione opción", 1, 3)
    
    search_fields = None
    if search_choice == 3:
        fields_input = safe_input("Ingrese nombres de campos (separados por comas): ", required=True)
        search_fields = [field.strip() for field in fields_input.split(',')]
    elif search_choice == 2:
        search_fields = ['filename']
    
    try:
        results = app.image_manager.search_images(query, search_fields)
        
        if not results:
            print("No se encontraron imágenes que coincidan con la consulta.")
            return
        
        print(f"\nSe encontraron {len(results)} imágenes coincidentes:")
        for i, image in enumerate(results, 1):
            print(f"{i}. {image.filename}")
            print(f"   ID: {image.image_id}")
            print(f"   Formato: {image.format_type}")
            print()
        
    except Exception as e:
        print(f"Error de búsqueda: {e}")

print("Funciones de eliminación y búsqueda de imágenes configuradas")

Funciones de eliminación y búsqueda de imágenes configuradas


In [ ]:
# Handle Menu Selection Logic - Función 6: Cargar Dataset de Zenodo
def load_dataset_menu():
    """Manejar la carga de datasets."""
    print("\n" + "======================")
    print("CARGAR DATASET")
    print("=" * 50)
    print("1. Descargar dataset de Zenodo")
    print("2. Cargar desde directorio local")
    print("3. Crear dataset de muestra")
    
    choice = validate_numeric_choice("Seleccione opción", 1, 3)
    
    try:
        if choice == 1:
            # Descargar desde Zenodo
            record_id = safe_input("Ingrese ID de registro de Zenodo (default: 7105232): ")
            if not record_id:
                record_id = "7105232"
            
            print("Descargando dataset... Esto puede tomar un tiempo.")
            dataset_path = app.data_loader.download_zenodo_dataset(record_id)
            
            # Preguntar si el usuario quiere registrar imágenes
            register = validate_yes_no_input("¿Registrar imágenes descargadas?")
            
            if register:
                bulk_register_images(dataset_path)
            
        elif choice == 2:
            # Cargar desde directorio local
            directory = safe_input("Ingrese ruta del directorio: ", required=True)
            
            if not os.path.exists(directory):
                print(f"Directorio no encontrado: {directory}")
                return
            
            image_files = app.data_loader.load_from_directory(directory)
            
            if not image_files:
                print("No se encontraron archivos de imagen soportados.")
                return
            
            print(f"Se encontraron {len(image_files)} archivos de imagen.")
            register = validate_yes_no_input("¿Registrar todas las imágenes?")
            
            if register:
                bulk_register_images(directory, image_files)
        
        elif choice == 3:
            # Crear dataset de muestra
            output_dir = safe_input("Directorio de salida (default: data/samples): ")
            if not output_dir:
                output_dir = "data/samples"
            
            num_samples_str = safe_input("Número de muestras (default: 10): ")
            try:
                num_samples = int(num_samples_str) if num_samples_str else 10
            except ValueError:
                num_samples = 10
            
            sample_files = app.data_loader.create_sample_dataset(output_dir, num_samples)
            
            if sample_files:
                print(f"Se crearon {len(sample_files)} archivos de muestra.")
                register = validate_yes_no_input("¿Registrar imágenes de muestra?")
                
                if register:
                    bulk_register_images(output_dir, sample_files)
                
    except Exception as e:
        print(f"Error cargando dataset: {e}")

def bulk_register_images(directory: str, file_list: Optional[List[str]] = None):
    """Registrar múltiples imágenes a la vez."""
    if file_list is None:
        file_list = app.data_loader.load_from_directory(directory)
    
    if not file_list:
        print("No hay archivos para registrar.")
        return
    
    print(f"\nRegistrando {len(file_list)} imágenes...")
    
    success_count = 0
    error_count = 0
    
    for i, file_path in enumerate(file_list, 1):
        try:
            print(f"Procesando {i}/{len(file_list)}: {os.path.basename(file_path)}")
            
            image_id = app.image_manager.register_image(
                file_path=file_path,
                load_pixel_data=False  # No cargar datos de píxeles para operaciones masivas
            )
            
            success_count += 1
            
        except Exception as e:
            print(f"  Error: {e}")
            error_count += 1
    
    print(f"\nRegistro masivo completado:")
    print(f"  Registrados exitosamente: {success_count}")
    print(f"  Errores: {error_count}")

print("Función de carga de datasets configurada")

Función de carga de datasets configurada


## 7. Create Main Menu Loop
Construimos el bucle principal del programa que muestra el menú, lee la entrada del usuario y procesa las selecciones hasta que el usuario elija salir.

In [ ]:
# Create Main Menu Loop - VERSIÓN MEJORADA CON BUCLE AUTOMÁTICO
def process_menu_choice(choice: str) -> bool:
    """
    Procesar la elección del menú del usuario.
    
    Args:
        choice: La opción seleccionada por el usuario
        
    Returns:
        bool: True si el programa debe continuar, False si debe salir
    """
    option = MENU_OPTIONS[choice]
    
    print(f"\nEjecutando: {option['title']}")
    print("=" * 60)
    
    try:
        if choice == '1':
            register_image_menu()
        elif choice == '2':
            view_images_menu()
        elif choice == '3':
            modify_metadata_menu()
        elif choice == '4':
            delete_image_menu()
        elif choice == '5':
            search_images_menu()
        elif choice == '6':
            load_dataset_menu()
        elif choice == '7':
            print("¡Gracias por usar el Sistema de Gestión de Imágenes Médicas!")
            print("Trabajo Final Grupo 4 - ¡Gracias Magister!")
            return False
            
    except KeyboardInterrupt:
        print("\n\nOperación interrumpida por el usuario.")
    except Exception as e:
        print(f"Error durante la operación: {e}")
        app.logger.error(f"Error de aplicación: {e}")
    
    return True

def run_menu_loop():
    """
    Ejecutar el bucle completo del menú hasta que el usuario elija salir.
    Esta función mantiene el menú activo hasta que se seleccione la opción 7.
    """
    print("\n¡Bienvenido al Sistema de Gestión de Imágenes Médicas!")
    print("El menú se mostrará automáticamente después de cada operación.")
    print("Seleccione la opción 7 para salir del sistema.")
    
    try:
        while True:
            # Mostrar menú y obtener elección
            choice = get_menu_choice()
            
            # Procesar la elección
            continue_program = process_menu_choice(choice)
            
            # Si el usuario eligió salir (opción 7), terminar el bucle
            if not continue_program:
                break
            
            # Pausa antes de mostrar el menú nuevamente
            print("\n" + "==============================" )
            print("Operación completada. Regresando al menú principal...")
            print("==============================")
            
            # Pequeña pausa para mejor experiencia de usuario
            import time
            time.sleep(1)
            
    except KeyboardInterrupt:
        print("\n\nAplicación interrumpida por el usuario. ¡Hasta luego!")
    except Exception as e:
        print(f"\nError de aplicación: {e}")

def execute_menu_option():
    """
    Función de conveniencia para ejecutar UNA opción del menú.
    Para el bucle completo, use run_menu_loop().
    """
    try:
        choice = get_menu_choice()
        continue_program = process_menu_choice(choice)
        
        if continue_program:
            print(f"\nOperación completada.")
            print("Ejecute 'run_menu_loop()' para el menú continuo o 'execute_menu_option()' para una sola operación.")
        else:
            print("\n¡Hasta luego!")
            
    except KeyboardInterrupt:
        print("\n\nAplicación interrumpida por el usuario. ¡Hasta luego!")
    except Exception as e:
        print(f"\nError de aplicación: {e}")

def run_interactive_menu():
    """
    Información de inicio del sistema.
    """
    print("\n¡Sistema de Gestión de Imágenes Médicas Listo!")
    print("Opciones disponibles:")
    print("   run_menu_loop()      - Ejecutar menú continuo (recomendado)")
    print("   execute_menu_option() - Ejecutar una sola opción")
    print("   display_main_menu()   - Solo mostrar el menú")

# Configurar el menú principal
run_interactive_menu()

print("Sistema de menú principal configurado y listo para usar")


¡Sistema de Gestión de Imágenes Médicas Listo!
Opciones disponibles:
   run_menu_loop()      - Ejecutar menú continuo (recomendado)
   execute_menu_option() - Ejecutar una sola opción
   display_main_menu()   - Solo mostrar el menú
Sistema de menú principal configurado y listo para usar


In [49]:
# ¡EJECUTE ESTA CELDA PARA COMENZAR EL MENÚ CONTINUO!
# ====================================================

# Iniciar el bucle del menú que se mantendrá activo hasta elegir salir (opción 7)
run_menu_loop()


¡Bienvenido al Sistema de Gestión de Imágenes Médicas!
El menú se mostrará automáticamente después de cada operación.
Seleccione la opción 7 para salir del sistema.

            MENU PRINCIPAL
1. Registrar Nueva Imagen
2. Ver Imágenes Registradas
3. Modificar Metadatos de Imagen
4. Eliminar Imagen
   -------------------------
5. Buscar Imágenes
6. Cargar Dataset de Zenodo
   -------------------------
7. Salir



Seleccione una opción (1-7):  1



Ejecutando: Registrar Nueva Imagen

REGISTRAR NUEVA IMAGEN


Ingrese la ruta del archivo de imagen:  C:\Users\EYNAR\Desktop\MAESTRAI_IA_MOD1\data\test\TestImage.jpg
¿Agregar metadatos personalizados? (s/n):  n
¿Cargar datos de píxeles en memoria? (s/n):  n


2025-10-19 19:01:00,330 - medical_image_manager.services.image_manager - INFO - Registered image: 6b612cdc-5b5d-40af-b7df-2b8950505533 (TestImage.jpg)


...

Confirmar eliminación (s/n):  s


2025-10-19 19:01:23,842 - medical_image_manager.services.image_manager - INFO - Deleted image: 5e23cdf1-57c4-4d41-b209-4ff6c4901011


Imagen eliminada exitosamente.

Operación completada. Regresando al menú principal...

            MENU PRINCIPAL
1. Registrar Nueva Imagen
2. Ver Imágenes Registradas
3. Modificar Metadatos de Imagen
4. Eliminar Imagen
   -------------------------
5. Buscar Imágenes
6. Cargar Dataset de Zenodo
   -------------------------
7. Salir



Seleccione una opción (1-7):  7



Ejecutando: Salir
¡Gracias por usar el Sistema de Gestión de Imágenes Médicas!
Trabajo Final Grupo 4 - ¡Gracias Magister!
